<table style="border: none" align="left">
    <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/pmservice/cars-4-you/master/static/images/logo.png" width="200" alt="Icon"></th>
       <th style="border: none"><font face="verdana" size="5" color="black"><b>Area and Action - AI function</b></th>
   </tr>
</table>


<img align=left src="https://github.com/pmservice/cars-4-you/raw/master/static/images/ai_function.png" alt="Icon" width="618"> 


AI function with logic to sequentially call two models: BUSINESS_AREA and ACTION.

Contents

- [0. Setup](#setup)
- [1. Introduction](#introduction)
- [2. Define AI function](#ai_function)
- [3. Store the AI function in the repository](#persistence)
- [4. Deploy AI function](#deploy)

<a id="setup"></a>
## 0. Setup

In this section please use below cell to upgrade the `watson-machine-learning-client`.

In [1]:
!rm -rf $PIP_BUILD/watson-machine-learning-client
!pip install --upgrade watson-machine-learning-client==1.0.260

Requirement already up-to-date: watson-machine-learning-client==1.0.260 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: lomond in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client==1.0.260)
Requirement not upgraded as not directly required: certifi in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client==1.0.260)
Requirement not upgraded as not directly required: pandas in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client==1.0.260)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client==1.0.260)
Requirement not upgraded as not directly required: tqdm in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client==1.0.260)
Requirement not upgraded as not directly r

<a id="introduction"></a>
## 1. Introduction

Models to predict Business Area and Action based on client feedback will be embeded into AI function. An example of payload logging using AI function will be introduced.

<a id="ai_function"></a>
## 2. Define AI function

In this section we define AI function and test it locally.

### Template

We start with template version of AI function to present its structure in simplest case.

    # Template version

    def score_template(payload):
        """AI function example.

        # Here we present data schema to be used by function:
        Example:
          {
            "fields": ["Gender", "Status", "Children", "Age", "Customer_Status"],
            "values": [
              ["Male", "M", 2, 48, "Inactive"],
              ["Female", "S", 0, 23, "Inactive"]
            ]
          }
        """
        fields = payload['fields'] + ['Prediction', 'Probability']
        values = [record + [int(record[0] == 'Male'), 0.9] for record in payload['values']]
        return { 'fields': fields, 'values': values }

    template_payload = {"fields": [],
                        "values": [["Male", "M", 2, 48, "Inactive"], ["Female", "S", 0, 23, "Inactive"]]}

    print(score_template(template_payload))

### Deployed models metadata extraction

We have to identify model deployments to be used before we define AI function. We extract scoring url and information about model version.

In [2]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

**Tip:** Put your Watson Machine Learning service credentials here.

In [3]:
# @hidden_cell
# How to get associated service credentials

wml_credentials = {
  "apikey": "***",
  "instance_id": "***",
  "password": "***",
  "url": "https://us-south.ml.cloud.ibm.com",
  "username": "***"
}

In [4]:
client = WatsonMachineLearningAPIClient(wml_credentials)

We list all deployed models predicting `Action` and `Business Area` to select one to be used in the AI function.

In [5]:
client.deployments.list()

------------------------------------  ---------------------------------------------------------  ------  --------------  ------------------------  --------------  ----------
GUID                                  NAME                                                       TYPE    STATE           CREATED                   FRAMEWORK       ASSET TYPE
f221801b-362e-48c1-8a5e-9850b4c57109  CARS4U - Satisfaction Prediction - AI Function Deployment  online  DEPLOY_SUCCESS  2018-07-31T13:48:12.424Z  n/a             function
95702a10-e783-410a-9c84-2f0499554296  CARS4U - Satisfaction Prediction Model Deployment          online  DEPLOY_SUCCESS  2018-07-31T13:47:49.984Z  tensorflow-1.5  model
7712d6b7-aca4-4b67-8d77-fbbce1680fc9  CARS4U - Action Model Deployment                           online  DEPLOY_SUCCESS  2018-07-31T13:45:11.869Z  mllib-2.1       model
4bb27d68-f2ba-4825-b451-c745450d83b4  CARS4U - Business Area Prediction Model Deployment         online  DEPLOY_SUCCESS  2018-07-31T13:43:31.2

We select deployed model to predict `Business Area` and model to recommend `Action`. We have to extract scoring endpoints for two models.

In [6]:
deployments = client.deployments.get_details()['resources']

for x in deployments:
    if (x['entity']['name'] == 'CARS4U - Action Model Deployment'):
        action_scoring_url = client.deployments.get_scoring_url(x)
    if (x['entity']['name'] == 'CARS4U - Business Area Prediction Model Deployment'):
        business_area_scoring_url = client.deployments.get_scoring_url(x)
        
print("Action url: ", action_scoring_url)
print("Business area url: ", business_area_scoring_url)

Action url:  https://us-south.ml.cloud.ibm.com/v3/wml_instances/aaed6937-c0e7-4307-8a17-361aca257c7e/deployments/7712d6b7-aca4-4b67-8d77-fbbce1680fc9/online
Business area url:  https://us-south.ml.cloud.ibm.com/v3/wml_instances/aaed6937-c0e7-4307-8a17-361aca257c7e/deployments/4bb27d68-f2ba-4825-b451-c745450d83b4/online


### AI function definition

We define AI function to be deployed in the IBM Cloud.

Put all parameters required to define AI function in dictionary.

In [7]:
ai_params = {"wml_credentials": wml_credentials, 
             "business_area_scoring_url": business_area_scoring_url, 
             "action_scoring_url": action_scoring_url}

In [8]:
def score_generator(params=ai_params):
    
    from watson_machine_learning_client import WatsonMachineLearningAPIClient

    wml_credentials = params["wml_credentials"]
    business_area_scoring_url = params["business_area_scoring_url"]
    action_scoring_url = params["action_scoring_url"]

    client = WatsonMachineLearningAPIClient(wml_credentials)

    def score(payload):
        """AI function with model version.

        Example:
          {"fields": ["ID", "Gender", "Status", "Children", "Age", "Customer_Status", "Car_Owner", "Customer_Service", "Satisfaction"],
           "values": [[2624, 'Male', 'S', 0, 49.27, 'Active', 'No', "Good experience with all the rental co.'s I contacted. I Just called with rental dates and received pricing and selected rental co.", 1]]}
        """
        
        scores_area = client.deployments.score(business_area_scoring_url, payload)
        idx_area = [i for i, x in enumerate(scores_area["fields"]) if (x == 'probability') or (x == 'predictedLabel')]

        fields_from_action = payload["fields"] + ["predictedLabel"]
        idx_from_action = set([i for i, x in enumerate(scores_area["fields"]) if x in fields_from_action])
        values_action = [[v for i, v in enumerate(rec) if i in idx_from_action] for rec in scores_area["values"]]
        
        payload_action = {"fields": payload["fields"] + ["Business_Area"], 
                          "values": values_action}
        
        scores_action = client.deployments.score(action_scoring_url, payload_action)
        idx_action = [i for i, x in enumerate(scores_action["fields"]) if (x == 'probability') or (x == 'predictedLabel')]

        values = [rec + [scores_area['values'][i][idx_area[0]], scores_area['values'][i][idx_area[1]], scores_action['values'][i][idx_action[0]], scores_action['values'][i][idx_action[1]]] for i, rec in enumerate(payload['values'])]

        fields = payload['fields'] + ['Probability_Area', 'Prediction_Area'] + ['Probability_Action', 'Prediction_Action']

        return {'fields': fields, 'values': values}

    return score

### AI function local test

Let's test defined AI function in notebook before we deploy it in the IBM Cloud. We start with preparing sample payload for `score` function.

In [9]:
sample_payload = {"fields": ["ID", "Gender", "Status", "Children", "Age", "Customer_Status", "Car_Owner", "Customer_Service", "Satisfaction"],
                  "values": [[2624, 'Male', 'S', 0, 49.27, 'Active', 'No', "Good experience with all the rental co.'s I contacted. I Just called with rental dates and received pricing and selected rental co.", 1]]}       

Let's test AI function locally before we deploy it to the IBM Cloud.

In [10]:
score = score_generator()
scores_ai = score(sample_payload)
print(scores_ai)

{'values': [[2624, 'Male', 'S', 0, 49.27, 'Active', 'No', "Good experience with all the rental co.'s I contacted. I Just called with rental dates and received pricing and selected rental co.", 1, [0.3574468085106383, 0.3191489361702128, 0.0425531914893617, 0.06808510638297872, 0.07234042553191489, 0.059574468085106386, 0.05106382978723404, 0.029787234042553193], 'Service: Knowledge', [1.0, 0.0, 0.0, 0.0, 0.0], 'NA']], 'fields': ['ID', 'Gender', 'Status', 'Children', 'Age', 'Customer_Status', 'Car_Owner', 'Customer_Service', 'Satisfaction', 'Probability_Area', 'Prediction_Area', 'Probability_Action', 'Prediction_Action']}


<a id="persistence"></a>
## 3. Store the AI function in the repository

In this section we store AI function to Watson machine Learning repository.

Required prerequisite to store AI function is runtime specification.

In [11]:
runtime_meta = {
            client.runtime_specs.ConfigurationMetaNames.NAME: "Basic runtime specification",
            client.runtime_specs.ConfigurationMetaNames.DESCRIPTION: "Runtime for AI function",
            client.runtime_specs.ConfigurationMetaNames.PLATFORM: {
               "name": "python",
               "version": "3.5"
             }}

In [12]:
runtime_details = client.runtime_specs.create(runtime_meta)
runtime_url = client.runtime_specs.get_url(runtime_details)
print(runtime_url)

https://us-south.ml.cloud.ibm.com/v4/runtimes/ae473ac0-c10e-413b-b349-1adbc5b1ab61


Now we are ready to save function to Watson Machine Learning repository.

In [13]:
meta_data = {
    client.repository.FunctionMetaNames.NAME: 'CARS4U - Business area and Action Prediction - AI Function',
    client.repository.FunctionMetaNames.RUNTIME_URL: runtime_url
}

function_details = client.repository.store_function(meta_props=meta_data, function=score_generator)

Recognized generator function.


In [14]:
ai_function_uid = client.repository.get_function_uid(function_details)
print(ai_function_uid)

4dbe7c73-9a1d-4162-96e9-6518b0781e19


<a id="deploy"></a>
## 4. Deploy AI function

In this section we deploy AI function in the IBM Cloud and test created deployment using sample payload.

In [15]:
function_deployment_details = client.deployments.create(ai_function_uid, "CARS4U - Business area and Action Prediction - AI Function Deployment")



#######################################################################################

Synchronous deployment creation for uid: '4dbe7c73-9a1d-4162-96e9-6518b0781e19' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS....
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='f466aea9-62f3-444a-a79a-5eff98053029'
------------------------------------------------------------------------------------------------




### AI function online deployment test

Now we are ready to test created deployment.

In [16]:
scoring_url = client.deployments.get_scoring_url(function_deployment_details)

In [17]:
scoring_results = client.deployments.score(scoring_url, sample_payload)
print(scoring_results)

{'values': [[2624, 'Male', 'S', 0, 49.27, 'Active', 'No', "Good experience with all the rental co.'s I contacted. I Just called with rental dates and received pricing and selected rental co.", 1, [0.3574468085106383, 0.3191489361702128, 0.0425531914893617, 0.06808510638297872, 0.07234042553191489, 0.059574468085106386, 0.05106382978723404, 0.029787234042553193], 'Service: Knowledge', [1.0, 0.0, 0.0, 0.0, 0.0], 'NA']], 'fields': ['ID', 'Gender', 'Status', 'Children', 'Age', 'Customer_Status', 'Car_Owner', 'Customer_Service', 'Satisfaction', 'Probability_Area', 'Prediction_Area', 'Probability_Action', 'Prediction_Action']}


---